<a href="https://colab.research.google.com/github/Pimateos/Fork/blob/master/deep_learning_solucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

# Deep Learning

Redes profundas, activaciones y métodos de inicialización de pesos

Vamos a cargar un dataset de datos MNIST, correspondiente a números manuscritos con unas dimensiones de 28x28 pixels. El dataset se divide en :


*   Conjunto de entrenamiento (train)
*   Conjunto de test(test)

A su vez, en ambos tenemos los valores x_ correspondientes a las variables e y_ correspondiente a la variable objetivo, que pretendemos predecir.




In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [14]:
np.prod(x_train.shape[1:])
len(x_train)


60000

TensorFlow sólo reconoce tensores de floats (vectores). Por esta razón, lo primero que se hace es transformarlo en un vector con elementos de tipo float32.  Adicionalmente, las imagenes (x_) son divididas por 255 para que todos los valores se encuentren entre 0 y 1. Se les modifica las dimensiones, aplanandolas, es decir, los 784(28x28) pixels aparecen para cada registro y transformando en la codificación one-hot las distintas posibles clasificaciones (0-9 números)

In [0]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Ahora vamos a crear la red neuronal densa. Para ello, definimos que tiene en su capa de entrada 784 variables de entrada. Después, tiene 4 layers:
1.- Una capa densa (todas las variables se conectan con todas las neuronas) con 100 neuronas, con una función de activación tangencial y un kerne

In [0]:
from keras.layers import Input, Dense
from keras.models import Model

x = Input(shape=(784,))
z = Dense(100, activation='tanh', kernel_initializer='random_uniform')(x)
z = Dense(100, activation='tanh', kernel_initializer='random_uniform')(z)
z = Dense(100, activation='tanh', kernel_initializer='random_uniform')(z)
y = Dense(10, activation='sigmoid')(z)

mlp = Model(x, y)

In [0]:
mlp.compile(optimizer='sgd', loss='categorical_crossentropy')

In [6]:
mlp.fit(x_train, y_train,
        epochs=10,
        batch_size=256,
        shuffle=True)

Epoch 1/10
60000/60000 [==============================] - 2s 35us/step - loss: 2.2910
Epoch 2/10
60000/60000 [==============================] - 2s 26us/step - loss: 2.2630
Epoch 3/10
60000/60000 [==============================] - 2s 25us/step - loss: 2.2088
Epoch 4/10
60000/60000 [==============================] - 1s 24us/step - loss: 2.0997
Epoch 5/10
60000/60000 [==============================] - 1s 24us/step - loss: 1.9362
Epoch 6/10
60000/60000 [==============================] - 1s 24us/step - loss: 1.7514
Epoch 7/10
60000/60000 [==============================] - 1s 24us/step - loss: 1.5722
Epoch 8/10
60000/60000 [==============================] - 1s 24us/step - loss: 1.3574
Epoch 9/10
60000/60000 [==============================] - 1s 24us/step - loss: 1.1445
Epoch 10/10
60000/60000 [==============================] - 1s 24us/step - loss: 0.9621


In [7]:
mlp.evaluate(x_test, y_test)

10000/10000 [==============================] - 0s 28us/step


0.8675141201019287